In [52]:
import collections

def leitura_fasta(nome_arquivo):
    'Processamento de arquivo fasta e retorno de dicionario com as sequencias lidas'
    
    sequencias = {}
    seq_id = ''
    
    try:
        arquivo_fasta =  open(nome_arquivo)
    except 'IOError':
       print('Arquivo nao encontrado!') 

    for linha in arquivo_fasta:
        if linha[0] == '>':
            seq_id = linha.rstrip()[1:linha.find(' ')]
            sequencias[seq_id] = ''
        elif linha != '':
            sequencias[seq_id] = sequencias[seq_id] + linha.rstrip()
    print('quantidade de sequencias lidas=>', len(sequencias))
    return sequencias

In [53]:
def obter_tamanho_sequencias(sequencias):
    'obtem um dicionario de sequencias e retorna o tamanho delas ordenado de forma descendente '
    tamanhos = {}
    for id_seq, sequencia in sequencias.items():
        tamanhos[id_seq] = len(sequencia)
    return  sorted(tamanhos.items(), key=lambda x: x[1], reverse=True)

In [54]:
def obter_fita_reversa_complementar(sequencia):
    pareamento_bases = {'A':'T', 'T':'A', 'C':'G', 'G':'C', 'N':'N'}
    sequencia_invertida = list(sequencia[::-1]) 
    complemento = [pareamento_bases[base] for base in sequencia_invertida]
    return ''.join(complemento)

In [68]:
def obter_qtd_bases(sequencia):
    count = collections.Counter()
    for seq in sequencia:
        count.update(seq)
    print(count)

In [69]:
def obter_orfs(sequencia, frame=1, reverso_complementar = False):
    'obtem as ORFs de uma sequencia em determinado frame (1,2 ou 3). O frame default é 1'
    start_codon  = 'ATG'
    stop_codons  = ['TAA', 'TAG', 'TGA']
    tamanho_codon= 3
    sequencia_frame = sequencia[frame-1:]
    
    if reverso_complementar:
        sequencia_frame =  obter_fita_reversa_complementar(sequencia_frame)
            
    orfs = []
    orf = ['' , '', '']
    
    for i in range (0, len(sequencia_frame)-tamanho_codon + 1, tamanho_codon):
        if sequencia_frame[i:i+3] == start_codon and orf[0] == '' :
            orf[0] = i + frame #pos start
        
        if sequencia_frame[i:i+3] in stop_codons and orf[0] != '':
            orf[1] = i + frame #pos stop
            orf[2] = (orf[1] + tamanho_codon) - orf[0] #tamanho 
            orfs.append(orf)
            orf = ['' , '', '']
            
    return orfs

In [70]:
'obter as orfs de cada sequencia ordenadas de forma descendente'
def obter_orfs_sequencias (sequencias, frame):
    seq_forward_orfs = {}
    for seq_id, sequencia in sequencias.items():
        orfs = obter_orfs(sequencia, frame )
        seq_forward_orfs[seq_id]=orfs
    return seq_forward_orfs

In [71]:
def encontrar_repeticoes_palavra(sequencia, palavra):
    'encontrar repetições da palavra'
    repeticoes_palavra = []
    for i in range (len(sequencia) - len(palavra) + 1):
        encontrou = True
        for j in range(len(palavra)):
            if sequencia[i+j] != palavra[j]:
                encontrou = False
                break
        if encontrou: 
            repeticoes_palavra.append(i)
        
    return repeticoes_palavra

In [72]:
def encontrar_palavras_sequencia(sequencia, tamanho):
    palavras = []
    for i in range(len(sequencia) - tamanho + 1):
        palavra = sequencia[i:i+tamanho]
        palavras.append(palavra)
    return palavras

In [73]:
'buscar palavras repetidas nas sequencias'
def encontrar_repeticoes_sequencia(sequencia, tamanho):
    palavra_repeticoes = {}
    palavras = encontrar_palavras_sequencia(sequencia, tamanho)
    if len(palavras) != 0:
        for palavra in palavras: 
            repeticoes = encontrar_repeticoes_palavra(sequencia, palavra) 
            palavra_repeticoes[palavra] = repeticoes
    return palavra_repeticoes  

In [74]:
def obter_rank_repeticoes(repeticoes_sequencias, min_ocorrencias):
    'obter palavras repetidas em comum nas sequencias'
    todas_palavras = []
    contagem_palavras = {}
    contagem_palavras_filtered = {}
    
    for seq_id, palavras in repeticoes_sequencias.items():
        for palavra, repeat in palavras.items():
            todas_palavras.append(palavra)
    todas_palavras = set(todas_palavras)
    
    'obter rank das palavras mais frequentes'
    for palavra in todas_palavras:
        contador = 0
        for seq_id, palavras in repeticoes_sequencias.items():     
            if palavra  in palavras:
                contador = contador + len(palavras[palavra])
                contagem_palavras[palavra] = contador
            
    'extrair apenas as repeticoes com um numero minimo de ocorrencias'
    for palavra, contagem in contagem_palavras.items():
        if contagem >= min_ocorrencias:
            contagem_palavras_filtered[palavra] = contagem
        
    return sorted(contagem_palavras_filtered.items(), key=lambda x: x[1], reverse=True)

In [75]:
'ler arquivo fasta'
arquivo = 'dna2.fasta'
sequencias = leitura_fasta(arquivo)

quantidade de sequencias lidas=> 18


In [76]:
obter_qtd_bases(sequencias['gi|142022655|gb|EQ086233.1|255'])

Counter({'G': 1676, 'C': 1626, 'T': 863, 'A': 729})


In [77]:
'obter o tamanho das sequencias'
tamanhos = obter_tamanho_sequencias(sequencias)
tamanhos

[('gi|142022655|gb|EQ086233.1|255', 4894),
 ('gi|142022655|gb|EQ086233.1|16', 4804),
 ('gi|142022655|gb|EQ086233.1|91', 4635),
 ('gi|142022655|gb|EQ086233.1|454', 4564),
 ('gi|142022655|gb|EQ086233.1|293', 4338),
 ('gi|142022655|gb|EQ086233.1|396', 4076),
 ('gi|142022655|gb|EQ086233.1|45', 3511),
 ('gi|142022655|gb|EQ086233.1|250', 2867),
 ('gi|142022655|gb|EQ086233.1|527', 2646),
 ('gi|142022655|gb|EQ086233.1|4', 2095),
 ('gi|142022655|gb|EQ086233.1|277', 1432),
 ('gi|142022655|gb|EQ086233.1|75', 1352),
 ('gi|142022655|gb|EQ086233.1|304', 1151),
 ('gi|142022655|gb|EQ086233.1|594', 967),
 ('gi|142022655|gb|EQ086233.1|584', 964),
 ('gi|142022655|gb|EQ086233.1|88', 890),
 ('gi|142022655|gb|EQ086233.1|322', 442),
 ('gi|142022655|gb|EQ086233.1|346', 115)]

In [78]:
'obtem as palavras do tamanho informado nas sequencias'
repeticoes_sequencias = {}
for seq_id, sequencia in sequencias.items():
    repeticoes = encontrar_repeticoes_sequencia(sequencia, 6)
    repeticoes_sequencias[seq_id] = repeticoes

In [79]:
'''obtem as palavras em comum mais frequentes (de acordo com o numero minimo de repeticoes informado)
e ordenadas de forma descendente '''
rank_palavras_6 = obter_rank_repeticoes(repeticoes_sequencias, 100)
rank_palavras_6

[('GCGCGC', 153),
 ('CGCGCG', 151),
 ('GCCGCG', 147),
 ('GCGCCG', 135),
 ('CGCGGC', 131),
 ('CGGCCG', 125),
 ('CGCCGC', 121),
 ('GCCGGC', 119),
 ('GCGCGG', 115),
 ('CGGCGC', 109),
 ('GCGGCG', 106),
 ('CCGCGC', 105),
 ('CGCCGA', 105),
 ('GCGACG', 104),
 ('CGCGCC', 103),
 ('GCGCGA', 102),
 ('CGCGAC', 100)]

In [80]:
'obter orfs do frame 1 forward de cada sequencia'
seq_forward_orfs_frame_1 = obter_orfs_sequencias(sequencias, 1)
seq_forward_orfs_frame_1

{'gi|142022655|gb|EQ086233.1|91': [[229, 904, 678],
  [979, 2272, 1296],
  [2386, 3091, 708],
  [3745, 4030, 288]],
 'gi|142022655|gb|EQ086233.1|304': [[520, 586, 69], [859, 961, 105]],
 'gi|142022655|gb|EQ086233.1|255': [[292, 1732, 1443],
  [1864, 2491, 630],
  [2503, 2566, 66],
  [2581, 2611, 33]],
 'gi|142022655|gb|EQ086233.1|45': [[385, 2776, 2394]],
 'gi|142022655|gb|EQ086233.1|396': [[145, 406, 264],
  [529, 1585, 1059],
  [2374, 3184, 813],
  [3649, 3745, 99]],
 'gi|142022655|gb|EQ086233.1|250': [[562, 2119, 1560], [2404, 2599, 198]],
 'gi|142022655|gb|EQ086233.1|322': [],
 'gi|142022655|gb|EQ086233.1|88': [[82, 199, 120],
  [316, 361, 48],
  [520, 571, 54],
  [733, 745, 15]],
 'gi|142022655|gb|EQ086233.1|594': [[28, 67, 42]],
 'gi|142022655|gb|EQ086233.1|293': [[22, 199, 180],
  [226, 340, 117],
  [355, 622, 270],
  [682, 712, 33],
  [763, 907, 147],
  [973, 1012, 42],
  [1021, 1039, 21],
  [1090, 1219, 132],
  [1390, 1699, 312],
  [1990, 2239, 252],
  [2965, 3061, 99],
  [313

In [81]:
'obter orfs do frame 2 forward de cada sequencia. Para cada orf são exibidas as posicoes de start, stop e o tamanho'
seq_forward_orfs_frame_2 = obter_orfs_sequencias(sequencias, 2)
seq_forward_orfs_frame_2

{'gi|142022655|gb|EQ086233.1|91': [[77, 125, 51],
  [293, 341, 51],
  [347, 353, 9],
  [512, 515, 6],
  [641, 653, 15],
  [659, 746, 90],
  [818, 1052, 237],
  [1103, 1133, 33],
  [1415, 1553, 141],
  [1658, 1664, 9],
  [1748, 1817, 72],
  [2078, 2213, 138],
  [2396, 2414, 21],
  [2558, 2582, 27],
  [2861, 2879, 21]],
 'gi|142022655|gb|EQ086233.1|304': [],
 'gi|142022655|gb|EQ086233.1|255': [[71, 155, 87],
  [206, 254, 51],
  [287, 293, 9],
  [431, 1613, 1185],
  [1856, 1865, 12],
  [2033, 2207, 177],
  [2378, 2510, 135],
  [2528, 3551, 1026],
  [3758, 4193, 438],
  [4217, 4496, 282]],
 'gi|142022655|gb|EQ086233.1|45': [[158, 683, 528],
  [1136, 1529, 396],
  [1613, 2048, 438],
  [2129, 2237, 111],
  [2426, 2522, 99],
  [2660, 2771, 114],
  [2834, 3251, 420],
  [3272, 3488, 219]],
 'gi|142022655|gb|EQ086233.1|396': [[185, 356, 174],
  [368, 530, 165],
  [734, 1280, 549],
  [1517, 2795, 1281],
  [2801, 3650, 852]],
 'gi|142022655|gb|EQ086233.1|250': [[104, 137, 36],
  [164, 713, 552],
 

In [82]:
'obter orfs do frame 3 forward de cada sequencia'
seq_forward_orfs_frame_3 = obter_orfs_sequencias(sequencias, 3)
seq_forward_orfs_frame_3

{'gi|142022655|gb|EQ086233.1|91': [[909, 1260, 354],
  [1401, 1437, 39],
  [2271, 2397, 129],
  [2856, 3441, 588],
  [3450, 3531, 84],
  [3543, 3690, 150],
  [3753, 3849, 99],
  [3942, 3954, 15],
  [4083, 4128, 48],
  [4263, 4290, 30],
  [4311, 4374, 66],
  [4392, 4470, 81]],
 'gi|142022655|gb|EQ086233.1|304': [[621, 765, 147],
  [909, 1026, 120],
  [1098, 1137, 42]],
 'gi|142022655|gb|EQ086233.1|255': [[777, 945, 171],
  [1179, 1185, 9],
  [1641, 1923, 285],
  [2022, 2031, 12],
  [2610, 2673, 66],
  [3705, 3903, 201],
  [4026, 4215, 192]],
 'gi|142022655|gb|EQ086233.1|45': [[216, 471, 258],
  [525, 555, 33],
  [699, 1020, 324],
  [2628, 2709, 84]],
 'gi|142022655|gb|EQ086233.1|396': [[429, 651, 225],
  [1941, 1968, 30],
  [1992, 2037, 48],
  [2391, 2472, 84],
  [2697, 2790, 96],
  [3120, 3180, 63],
  [3204, 3381, 180],
  [3399, 3411, 15],
  [3420, 3489, 72],
  [3495, 3540, 48],
  [3873, 4041, 171]],
 'gi|142022655|gb|EQ086233.1|250': [[246, 342, 99], [1374, 1620, 249]],
 'gi|142022655